# **Stacking from scratch**

Berikan penjelasan terkait Stacking!

Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [4]:
# Definisikan class stack
class Stack:
    def __init__(self, base_models, meta_model, n_folds=5, random_state=42):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.random_state = random_state

    def fit(self, X, y):
        self.base_models_ = [list() for _ in self.base_models]
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            for train_idx, holdout_idx in kf.split(X, y):
                instance = clone(model)
                instance.fit(X[train_idx], y[train_idx])
                y_pred = instance.predict(X[holdout_idx])
                out_of_fold_predictions[holdout_idx, i] = y_pred
                self.base_models_[i].append(instance)

        self.meta_model_ = clone(self.meta_model)
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    def predict(self, X):
        """Prediksi akhir dengan base models dan meta model"""
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)


### **Upload datasets**

In [2]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Pisahkan fitur dan target
X1 = glass_df.drop(columns=['target'])
y1 = glass_df['target']

# Encode target (optional, untuk case multiclass stacking)
encoder = LabelEncoder()
y1 = encoder.fit_transform(y1)

# Split data menjadi train dan test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)

# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [3]:
# StackingClassifier dan Blending Classifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Buat base models (model dasar)
estimators = [
    ('dt', DecisionTreeClassifier(random_state=0)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('svm', SVC(probability=True, random_state=0))
]

# Buat model meta (model akhir)
meta_model = LogisticRegression()

# =============================
# STACKING CLASSIFIER
# =============================
stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    stack_method='predict_proba',   # sesuai saran dari teks kamu
    passthrough=False,
    cv=5
)

# Latih model stacking
stack_model.fit(X1_train, y1_train)
y_pred_stack = stack_model.predict(X1_test)
acc_stack = accuracy_score(y1_test, y_pred_stack)

print(f"Accuracy StackingClassifier: {acc_stack:.4f}")

# =============================
# BLENDING CLASSIFIER (Voting)
# =============================
blend_model = VotingClassifier(
    estimators=estimators + [('lr', meta_model)],
    voting='soft'
)

# Latih model blending
blend_model.fit(X1_train, y1_train)
y_pred_blend = blend_model.predict(X1_test)
acc_blend = accuracy_score(y1_test, y_pred_blend)

print(f"Accuracy BlendingClassifier: {acc_blend:.4f}")


Accuracy StackingClassifier: 0.9737
Accuracy BlendingClassifier: 0.9737


**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [6]:
# StackingClassifier (scikit-learn)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# sk_estimators
sk_estimators = [
    ('dt', DecisionTreeClassifier(random_state=0)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('svm', SVC(probability=True, random_state=0))
]

# Final estimator
final_estimator = LogisticRegression()

# Beberapa metode stacking yang bisa dicoba
stack_methods = ['auto', 'predict_proba', 'predict']

# stack_methods
for i, method in enumerate(stack_methods):
    print(f"\n=== Metode ke-{i+1}: stack_method='{method}' ===")

    model = StackingClassifier(
        estimators=sk_estimators,
        final_estimator=final_estimator,
        stack_method=method,
        cv=5,
        passthrough=False
    )


=== Metode ke-1: stack_method='auto' ===

=== Metode ke-2: stack_method='predict_proba' ===

=== Metode ke-3: stack_method='predict' ===
